![train](http://cliparting.com/wp-content/uploads/2016/06/Train-clipart-for-kids-free-free-clipart-images.gif)

In [1]:
import numpy as np
np.random.seed(1019)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import lightgbm as lgb

import sklearn
from sklearn.model_selection import train_test_split 

import sys, os, gc, types
import time
from subprocess import check_output

In [2]:
sys.path.append('./utils')

from training import cv, train
from plotting import plot_importance
from data import Data

In [3]:
root_paths = [
    "/data/kaggle-instacart",
    "/Users/jiayou/Dropbox/珺珺的程序/Kaggle/Instacart",
    "/Users/jiayou/Dropbox/Documents/珺珺的程序/Kaggle/Instacart"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break

# Hyper-Parameter Search

In [4]:
name = 'v12-r0'
down_sample = None
test_size = 0.2
num_searches = 1
boosting_rounds = 10000
stopping_rounds = 10
aug = True

-----------

In [5]:
X_train, X_val, y_train, y_val = Data.train_val(down_sample=down_sample, test_size=test_size, aug=aug)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jiayou/Dropbox/珺珺的程序/Kaggle/Instacart/abt-share/abt_train.sb.orig1.pkl'

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

gc.collect()

In [ ]:
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_leaves': 256,
    'min_sum_hessian_in_leaf': 20,
    'max_depth': 12,
    'learning_rate': 0.05,
    'feature_fraction': 0.6,
    # 'bagging_fraction': 0.9,
    # 'bagging_freq': 3,
    'verbose': 1
}

In [ ]:
results = []
for i in range(num_searches):
    print('Start training...')
    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=boosting_rounds,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=stopping_rounds)

    print('Save model...')
    # save model to file
    gbm.save_model('model.{}.txt'.format(name))

    print('Start predicting...')
    
    print('Feature names:', gbm.feature_name())
    print('Feature importances:', list(gbm.feature_importance()))

----

In [ ]:
# predict
X_test = Data.test(down_sample=down_sample)
test = X_test.drop(['eval_set', 'order_id', 'reordered', 'product_id'], axis=1)

pred_t = gbm.predict(test, num_iteration=gbm.best_iteration)
pred_t_df = pd.DataFrame({'order_id':X_test.order_id,'product_id':X_test.product_id, 'reordered':pred_t})
pred_t_df.to_csv(
    os.path.join(root, 'test_prediction.{}.csv'.format(name)), index=False)